In [ ]:
from pathlib import Path

import torch
from sentence_transformers import SentenceTransformer
from torch import nn


class MessageClassifier(nn.Module):
    def __init__(self, input_size: int, hidden_sizes: list[int], output_size: int, dropout: float = 0.75):
        super().__init__()  # type: ignore
        self.activation = nn.ReLU()
        layers: list[nn.Module] = []
        current_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))

        self.model: nn.Sequential = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        return self.model(x)


class Classifier:
    def __init__(
        self, path_to_state_dict: Path, embedding_model: str = "intfloat/multilingual-e5-large-instruct"
    ) -> None:
        self.device: str = "cuda" if torch.cuda.is_available() else "cpu"
        self.embedding_model: SentenceTransformer = SentenceTransformer(embedding_model, device=self.device)

        self.classifier = MessageClassifier(1024, [48, 24], 2, dropout=0.75)
        self.classifier.to(self.device)
        self.classifier.load_state_dict(torch.load(path_to_state_dict, weights_only=True))
        self.classifier.eval()

    def predict(self, message: str) -> float:
        # Classifies a message as human (negative score) or bot (positive score).
        embedding: torch.Tensor = self.embedding_model.encode([message], convert_to_tensor=True)  # type: ignore
        # embedding.shape = (1, 1024)
        pred: torch.Tensor = self.classifier(embedding)
        # pred.shape = (1, 2)
        pred: torch.Tensor = pred.squeeze()
        # pred.shape = (2,)
        return torch.tanh(pred[1] - pred[0]).item()

In [22]:
# message_classifier = Classifier(os.path.join("classifier", "classifier_state_dict.pth"))
WEIGHTS_PATH = Path(".").absolute() / "classifier_state_dict.pth"
classifier: Classifier = Classifier(path_to_state_dict=WEIGHTS_PATH)

c:\Users\bergsmann\code\turing-game\fourmind\.venv\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
import json

DATA_PATH = Path(".").absolute() / "player_messages.json"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    data: dict[str, list[str]] = json.load(f)
data.keys()

dict_keys(['AllTalker', 'fourminds', 'Human'])

In [30]:
player_scores: dict[str, list[float]] = data.copy()  # type: ignore
for player_name in data.keys():
    player_scores[player_name] = []

In [ ]:
from tqdm import tqdm

for player_name, messages in data.items():
    for message in tqdm(messages):
        score = classifier.predict(message)
        player_scores[player_name].append(score)
    print(
        f"{player_name}: {len(messages)} messages, avg score: {sum(player_scores[player_name]) / len(messages):.3f}"  # noqa: E501
    )  # noqa: E501

100%|██████████| 1109/1109 [03:14<00:00,  5.71it/s]


AllTalker: 1109 messages, avg score: 0.222


100%|██████████| 876/876 [03:20<00:00,  4.37it/s]


fourminds: 876 messages, avg score: 0.014


100%|██████████| 738/738 [02:37<00:00,  4.69it/s]

Human: 738 messages, avg score: -0.641


In [33]:
with open("player_message_scores.json", "w", encoding="utf-8") as f:
    json.dump(player_scores, f, ensure_ascii=False, indent=2)